In [1]:
import pandas as pd # for reading csv
import numpy as np
import subprocess # for calling cutadapt
import regex
from Bio.Seq import Seq

In [2]:
%config Completer.use_jedi = False

In [3]:
def quickshell(command, print_output=True, output_path=None, return_output=False):
    process_output = subprocess.run(command, shell = True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout = process_output.stdout.decode('utf-8')
    stderr = process_output.stderr.decode('utf-8')
    output_string = 'STDOUT:\n%s\nSTDERR:\n%s\n'%(stdout,stderr)
    if print_output:
        print('$ %s'%command)
        print(output_string)
    if output_path is not None:
        with open(output_path,'w') as f:
            f.write(output_string)
    if return_output:
        return stdout, stderr

## pipeline to split reads by i7 and ezpool

In [25]:
import pandas as pd # for reading csv
import numpy as np
import subprocess # for calling cutadapt
import regex
from Bio.Seq import Seq
############
## INPUTS ##
############
# arguments
ezpool_barcode_errors_allowed = 1
threads = 6
minimum_trimmed_length = 10
# directory of input fastq files
input_directory = '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/fastq/'
# directory to output files
output_directory = '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/'
# tables of input file names, output file information, and adapter identities
i7_table = pd.read_csv('i7_io_novaseq.csv')
ezpool_table = pd.read_csv('ezpool_io.csv')
# additional multiplexing and trimming args
multiplexing_args = '--no-indels'
trimming_args = '-U 1' # remove first base from read 2, this is the invariant T (should not be included in barcoding)
ezpool_remove_t = True
print_cutadapt = False
############
############

# make a list to store dataframes of adapter information
output_dataframes = []

## BARCODE IDENTIFICATION AND FILE SPLITTING ##
# generate fasta file for ezpool adapter identification
if ezpool_remove_t is False:
    with open('ezpool_split.fasta','w') as f:
        for title,seq in zip(ezpool_table.ezpool, ezpool_table.sequence):
            f.write('>%s\n^%s\n'%(str(title),str(seq)))
if ezpool_remove_t is True:
    with open('ezpool_split.fasta','w') as f:
        for title,seq in zip(ezpool_table.ezpool, ezpool_table.sequence):
            f.write('>%s\n^%s\n'%(str(title),str(seq)[:-1]))
# iterate through i7 table to find split fastqs
for i in i7_table.iterrows():
    _, i7_data = i
    # input pair - R2 is first here as it contains the barcode as the first few nucleotides
    input_pair = [input_directory+str(i7_data.r2), input_directory+str(i7_data.r1)]
    # output pair
    output_pair = [output_directory+'_%s.R2.%s.{name}.fastq.gz'%(str(i7_data.title),str(i7_data.i7)),
                  output_directory+'_%s.R1.%s.{name}.fastq.gz'%(str(i7_data.title),str(i7_data.i7))]
    # prepare cutadapt command
    command = 'cutadapt %s -e %i -j %i -g file:%s -o %s -p %s %s %s'%(multiplexing_args,
                                                                      ezpool_barcode_errors_allowed, threads,
                                                                      'ezpool_split.fasta',
                                                                      output_pair[0], output_pair[1], 
                                                                      input_pair[0], input_pair[1])
    # run cutadapt to identify reads
    output_split, _ = quickshell(command, print_output = print_cutadapt, return_output = True)
    # print number of reads with adapter found
    print('In %s and mate:\n'%(input_pair[0].split('/')[-1]),'  ',
          (' '.join([m for m in regex.finditer('Read 1 with adapter:.*\n',output_split)][0].group()[:-1].split())).replace('1','2'))
    # record counts of each ezpool adapter found
    ezpool_counts = [int(m.group().split(' ')[1]) for m in regex.finditer('Trimmed: \d* times',output_split)]
    # generate a dataframe and append it to the list
    df = pd.DataFrame(columns = ['title', 'i7', 'ezpool', 'ezpool_counts'],
                         data = np.asarray([['%s-%s'%(i7_data.title,t) for t in ezpool_table.title.values],
                                            [i7_data.i7 for i in ezpool_counts],
                                            ezpool_table.ezpool.values,
                                            ezpool_counts]).T)
    output_dataframes.append(df)
# concatenate output dataframes into output_info - a single dataframe
output_info = pd.concat(output_dataframes,axis=0)

## FRAGMENT 5'- AND 3'-END ADAPTER TRIMMING ##
# generate a fasta file for ezpool barcode side adapter trimming
with open('ezpool_trim.fasta','w') as f:
    for title,seq in zip(ezpool_table.ezpool, ezpool_table.sequence):
        f.write('>%s\n%sAGATCGGAAGAGCACACGTCTGAACTCCAGTCAC\n'%(title,str(Seq(seq).reverse_complement())))
# storage for number of reads below minimum_trimmed_length
too_short_list = []
# iterate through all split fastq file pairs
for row in output_info.iterrows():
    _, sample_data = row
    # input file names do not have complete titles (only the i7 half)
    input_pair = ['%s_%s.R1.%s.%s.fastq.gz'%(output_directory,sample_data.title.split('-')[0],sample_data.i7,sample_data.ezpool),
                  '%s_%s.R2.%s.%s.fastq.gz'%(output_directory,sample_data.title.split('-')[0],sample_data.i7,sample_data.ezpool)]
    # output files should have complete titles: i7_title-ezpool_title
    output_pair = ['%s%s.R1.%s.%s.fastq.gz'%(output_directory,sample_data.title,sample_data.i7,sample_data.ezpool),
                   '%s%s.R2.%s.%s.fastq.gz'%(output_directory,sample_data.title,sample_data.i7,sample_data.ezpool)]
    # prepare cutadapt command
    command = 'cutadapt %s --overlap=1 --minimum-length=%i -j %i -a file:%s -A CCCATTCACTCTGCGTTGATACCACTGCTT -o %s -p %s %s %s'%(
                   trimming_args, minimum_trimmed_length, threads, 'ezpool_trim.fasta', output_pair[0], output_pair[1], input_pair[0], input_pair[1])
    # run cutadapt
    output_trim, _ = quickshell(command, print_output = print_cutadapt, return_output = True)
    # parse output to pull number of reads below minimum_trimmed_length
    too_short = int(regex.search('too short:\s*\S*',output_trim).group().split(' ')[-1].replace(',',''))
    too_short_list.append(too_short)
# add number of reads removed during trimming to the output_info dataframe
output_info.loc[:,'too_short'] = too_short_list
# remove split but untrimmed .fastq.gz files
command = 'rm %s_*fastq.gz'%output_directory
quickshell(command, print_output=False)
# output information about the fastq splits
output_info.to_csv('%sfastq_split_info.csv'%output_directory)

In 210310LauA_D21-2416-R2_sequence.fastq.gz and mate:
    Read 2 with adapter: 390,272,222 (97.4%)
In 210310LauA_D21-2417-R2_sequence.fastq.gz and mate:
    Read 2 with adapter: 429,486,022 (97.2%)
In 210310LauA_D21-2418-R2_sequence.fastq.gz and mate:
    Read 2 with adapter: 66,624,263 (96.9%)
In 210310LauA_D21-2419-R2_sequence.fastq.gz and mate:
    Read 2 with adapter: 60,436,502 (97.2%)
In 210310LauA_D21-2420-R2_sequence.fastq.gz and mate:
    Read 2 with adapter: 96,362,053 (90.5%)
This is cutadapt 3.2 with Python 3.8.5
Command line parameters: -U 1 --overlap=1 --minimum-length=10 -j 6 -a file:ezpool_trim.fasta -A CCCATTCACTCTGCGTTGATACCACTGCTT -o /home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/rz_frg_5m-ev1.R1.1.1.fastq.gz -p /home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/rz_frg_5m-ev1.R2.1.1.fastq.gz /home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/_rz_frg_5m.R1.1.1.fastq.gz /home/pculviner/notebook/storage/rnaseq

## pipeline to map ezpool reads with bowtie2

**inputs**
- `fastq.gz` files already split by i7 and ezpool indexes in the form:
		`[title].R[1/2].[i7].[ezpool].fastq.gz` where `title` is a concatenated title string (with `-` delimiter)
- a `csv` file with the key columns:
	1. `title`: concatenated i7/ezpool title string
	2. `i7`:  i7 index number
	3. `ezpool`: ezpool index number
- a genome to map to in the `genbank` format

**outputs**
- indexed `bam` files in the same directory as the starting `fastq` files
- a `csv` file with the the input columns and the following appended columns from parsed `bowtie2` output:
	1. `match_1`: percent of pairs matching exactly once
	2. `match_multiple`: percent of pairs matching multiple times
	3. `overall_align`: overall alignment rate

In [27]:
import Bio.SeqIO as SeqIO
import os
import subprocess
import pandas as pd

############
## INPUTS ##
############
# arguments
bowtie_args = '--very-sensitive'
threads = 7
maximum_fragment_size = 800
# info file
info_df = pd.read_csv('/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/fastq_split_info.csv',index_col=0)
# define directory with fastq files
working_directory = '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/'
# define sequence record to map to
sequence_record = '/home/pculviner/notebook/genomes/mg1655_v3/Ec_mg1655v3.gb'
sequence_record_title = 'mg1655v3'
# verbose output?
verbose = False

############
############
# define functions
def bowtie2_ezpool(fastq_paths=None, output_name=None, sequence_records=None, record_names=None,
                   maximum_fragment_size=800, threads=6,
                   additional_args='--very-sensitive', bowtie_path = '', verbose = False):
    # get current working directory (will return to after we're done)
    starting_dir = os.getcwd()
    # generate a temporary directory for bowtie to work in
    quickshell('mkdir %s_bowtietemp'%(output_directory), verbose)
    # move to temporary directory
    os.chdir('%s_bowtietemp'%(output_directory))
    # write fasta files for each genome record
    for record,record_name in zip(sequence_records, record_names):
        loaded_record = SeqIO.read(record,'genbank')
        SeqIO.write(loaded_record, record_name, 'fasta')
    # index the fasta files
    quickshell('%sbowtie2-build -f %s bowtie2_index'%(bowtie_path,','.join(record_names)), verbose)
    # run bowtie2 -> output alignment.sam
    _, alignment_output = quickshell('%sbowtie2 %s -p %i -X %i -x bowtie2_index -1 %s -2 %s -S alignment.sam'%(
                                     bowtie_path, additional_args, threads, maximum_fragment_size, fastq_paths[0], fastq_paths[1]),
                                     verbose, return_output = True)
    # sort .sam file -> .bam output
    quickshell('samtools sort -@ %i -l 9 -o %s alignment.sam'%(threads, output_name), verbose)
    # index the .bam file generating a .bam.bai in the same folder
    quickshell('samtools index -@ %i %s'%(threads, output_name), verbose)
    # return to starting directory and remove temp directory and files 
    os.chdir(starting_dir)
    quickshell('rm -r %s_bowtietemp'%(output_directory), verbose)
    return alignment_output

# storage for outputs
match_once = []
match_multiple = []
overall_align = []
# iterate across info rows
for row in info_df.iterrows():
    _, fastq_data = row
    # generate input fastqs from row data
    input_fastqs = ['%s%s.R1.%s.%s.fastq.gz'%(working_directory,fastq_data.title,fastq_data.i7,fastq_data.ezpool),
                    '%s%s.R2.%s.%s.fastq.gz'%(working_directory,fastq_data.title,fastq_data.i7,fastq_data.ezpool)]
    # generate output name from input fastq
    bam_output_name = input_fastqs[0].replace('.R1','').replace('.fastq.gz','.bam')
    # row data
    alignment_summary = bowtie2_ezpool(fastq_paths=input_fastqs, output_name=bam_output_name,
                                       sequence_records=[sequence_record], record_names=[sequence_record_title],
                                       maximum_fragment_size=maximum_fragment_size, threads=threads, additional_args=bowtie_args,
                                       verbose=verbose)
    # parse bowtie2 summary
    match_once.append(float(regex.search('\(\S*%\) aligned concordantly exactly', alignment_summary).group().split('%)')[0][1:]))
    match_multiple.append(float(regex.search('\(\S*%\) aligned concordantly >1', alignment_summary).group().split('%)')[0][1:]))
    overall_align.append(float(regex.search('\S*% overall', alignment_summary).group().split('%')[0]))
    
# store alignment information
info_df.loc[:,'match_1'] = match_once
info_df.loc[:,'match_multiple'] = match_multiple
info_df.loc[:,'overall_align'] = overall_align
# output information about the fastq splits
info_df.to_csv('%sbowtie2_mapping_info.csv'%working_directory)

## pipeline to map ezpool reads to regions + region quality control

In [28]:
import numpy as np
import pandas as pd
import pysam

############
## INPUTS ##
############
# info file
mapping_df = pd.read_csv(
    '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/bowtie2_mapping_info.csv',index_col=0)
mapping_df.index = np.asarray(range(len(mapping_df)))
# define directory with bam files
working_directory = '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/'
# input path to gene table
gene_table_path = '/home/pculviner/notebook/genomes/mg1655_v3/gt_mg1655_v3.csv'
# gene type column name
type_column = 'locus_type'

############
############
# define functions
def mapregioncounts_ezpool(path_to_bam, regions, mapq_cutoff=2, refseq_index=0):
    # regions must be in form [[strand, left, right], ....]
    # define a filter to only consider half of reads (left-most read)
    def filterReads(reads):
        for read in reads: # yield only reads which are proper paired, meet mapq cutoff, and are one side of the fragment
            if read.is_proper_pair and read.mapping_quality >= mapq_cutoff and not read.is_reverse:
                yield read
    # do the magic....
    bam = pysam.Samfile(path_to_bam, "rb") # load the bam file
    out_counts = np.zeros(len(regions))
    aligned_sizes = [] # record aligned region size
    left_region_pointer  = 0 # where in the reference the most recent read was
    regioned_fragments   = 0 # fragments overlapping with regions
    unregioned_fragments = 0 # fragments not overlapping with regions
    multi_region_fragments = 0 # fragments overlapping with multiple regions, resulting in double-counts
    filtered_reads = filterReads(bam.fetch(bam.references[refseq_index])) # get reads defined by filter function above
    for read in filtered_reads:
        aligned_sizes.append(read.template_length) # record aligned size to template
        # determine strand
        if read.is_read1:
            strand = 0 # positive strand
        else:
            strand = 1 # negative strand
        left = read.pos
        right = read.pos+read.template_length-1
        found_region = 0 # flag to determine if read ever mapped to a region
        # if there are no regions left to consider, the read is automatically unregioned
        if regions[-1,2] < left:
            unregioned_fragments += 1
            continue
        ## see if we should advance the region pointer
        # if left position of read is greater than right edge of read, we should stop considering these regions
        while regions[left_region_pointer,2] < left: 
            left_region_pointer += 1 # stop looking at region, its time has passed....
        # now try to compare the read to the remaining regions to check for overlap
        right_region_pointer = left_region_pointer
        while regions[right_region_pointer,1] <= right:
            if (strand == regions[right_region_pointer,0] and
                    right >= regions[right_region_pointer,1] and
                    left <= regions[right_region_pointer,2]): # check for overlap, if true, add a count
                out_counts[right_region_pointer] += 1
                found_region += 1
            right_region_pointer += 1
            if right_region_pointer >= len(regions): # catch the right pointer before it goes beyond availible regions
                break
        if found_region == 1:
            regioned_fragments+=1
        elif found_region > 1:
            multi_region_fragments += 1
        else:
            unregioned_fragments+=1
    return out_counts, regioned_fragments, unregioned_fragments, multi_region_fragments, aligned_sizes

## PREPARE GENE TABLE FOR USE ##
# load the gene table from the path
gene_table = pd.read_csv(gene_table_path, index_col=0)
# for QC, add inverse entries to this table
# convert gene types column to strings to avoid issues with undefined gene types
gene_table.loc[:,type_column] = gene_table.loc[:,type_column].values.astype(str)
# make an inverse gene table where strands are swapped, edit gene names and types to add "_inverse"
inv_gene_table = gene_table.copy()
inv_gene_table.strand = (~inv_gene_table.strand.values.astype(bool)).astype(int) # swap strands
inv_gene_table.index = [i+'_inverse' for i in inv_gene_table.index] 
inv_gene_table.loc[:,type_column] = [i+'_inverse' for i in inv_gene_table.loc[:,type_column]]
# generate a summary table for qc combining the inverse and correct strand tables
qc_gene_table = pd.concat([gene_table, inv_gene_table], axis=0).sort_values('start')
# get all gene type labels from gene_table
type_labels = qc_gene_table.loc[:,type_column].unique()
# generate label masks for QC table
type_label_masks = [qc_gene_table.loc[:,type_column].values == t for t in type_labels]
# generate mask for output of real genes (does not include those with the _inverse labels)
gene_table_output_mask = qc_gene_table.index.isin(gene_table.index)

## ITERATE ACROSS BAM FILES ##
# for storage of QC summary outputs
qc_outputs = []
# for storage of read count data
gene_outputs = []
# iterate across dataset rows to get individual bam files
for row in mapping_df.iterrows():
    _, exp_data = row
    # generate input fastqs from row data
    bam_path = '%s%s.%s.%s.bam'%(working_directory,exp_data.title,exp_data.i7,exp_data.ezpool)
    
    ## GET FRAGMENT CROSSING DATA FOR ALL REGIONS ##
    # get strand, start, and end for all regions in the expanded qc gene table
    all_regions = qc_gene_table.loc[:,['strand','start','end']].astype(int).values
    # map fragments to regions
    out_counts, regioned_fragments, unregioned_fragments, multi_region_fragments, aligned_sizes = (
                    mapregioncounts_ezpool(bam_path, all_regions, mapq_cutoff=0, refseq_index=0))
    
    ## STORE QC DATA ##
    # get summary values for various read statistics - these will be added to the dataframe and outputted to csv
    qc_summary_values = np.r_[np.asarray([np.sum(out_counts[mask]) for mask in type_label_masks]).astype(int), # values for each mask
                          np.mean(aligned_sizes), np.percentile(aligned_sizes,[5,10,50,90,95])]
    qc_outputs.append(qc_summary_values)
    
    ## STORE GENE-BY-GENE READCOUNT DATA ##
    counts_per_region = pd.DataFrame(data = out_counts[gene_table_output_mask].astype(int), 
                                     index = qc_gene_table.index[gene_table_output_mask],
                                     columns = ['%s.%s.%s'%(exp_data.title,exp_data.i7,exp_data.ezpool)])
    gene_outputs.append(counts_per_region)

## OUTPUT QC CSV ##
# generate column labels for the summary outputs
column_labels = np.r_[type_labels, ['mean', '5th', '10th', '50th', '90th', '95th']]
# append summary information to the mapping dataframe
read_summary_df = pd.concat([mapping_df,pd.DataFrame(data = np.asarray(qc_outputs), columns = column_labels)],
                             axis=1)
# output read summary dataframe
read_summary_df.to_csv('%smapping_summary.csv'%(working_directory))

## OUTPUT GENE COUNT CSV ##
gene_count_df = pd.concat(gene_outputs, axis = 1)
gene_count_df.to_csv('%sgene_counts.csv'%(working_directory))

## map reads to density maps

In [6]:
import regex
import pysam

def mapfragdensity_ezpool(path_to_bam, min_mapq=2, refseq_index=0, dtype=np.uint32):
    def filterReads(reads):
        for read in reads: # yield only reads which are proper paired, meet mapq cutoff, and are one side of the fragment
            if read.is_proper_pair and read.mapping_quality >= min_mapq and not read.is_reverse:
                yield read
    # prepare the output array and load the experiment
    bam = pysam.Samfile(path_to_bam, "rb") # load the bam file
    density_array = np.zeros((2,bam.lengths[refseq_index]), dtype)
    # load and filter the reads and add them to the density array
    filtered_reads = filterReads(bam.fetch(bam.references[refseq_index]))
    for read in filtered_reads:
        if read.is_read1: # maps to the plus strand since we're avoiding reverse reads and read1 is the read itself
            density_array[0,read.pos:read.pos+read.template_length] += 1
        else: # maps to the minus strand
            density_array[1,read.pos:read.pos+read.template_length] += 1
    return density_array

def map5primeends_ezpool(path_to_bam, min_mapq=2, refseq_index=0, dtype=np.uint32):
    """ Map 5' ends of reads generated using a mono-phosphate ligation strategy. """
    def filterReads(reads):
        for read in reads: # yield only reads which are proper paired, meet mapq cutoff, and are one side of the fragment
            if read.is_proper_pair and read.mapping_quality >= min_mapq and not read.is_read2:
                yield read
    # prepare the output array and load the experiment
    bam = pysam.Samfile(path_to_bam, "rb") # load the bam file
    density_array = np.zeros((2,bam.lengths[refseq_index]), dtype)
    # load and filter the reads and add them to the density array
    filtered_reads = filterReads(bam.fetch(bam.references[refseq_index]))
    for read in filtered_reads:
        try:
            if read.is_reverse: # maps to the minus strand
                density_array[1, read.pos+np.abs(read.alen)-1] += 1
            else: # maps to the plus strand
                density_array[0, read.pos] += 1
        except: # some will map to a point beyond the extent of the genome - this is of course problematic!
            pass
    return density_array

def map3primeends_ezpool(path_to_bam, min_mapq=2, refseq_index=0, dtype=np.uint32):
    """ Map 5' ends of reads generated using a mono-phosphate ligation strategy. """
    def filterReads(reads):
        for read in reads: # yield only reads which are proper paired, meet mapq cutoff, and are one side of the fragment
            if read.is_proper_pair and read.mapping_quality >= min_mapq and not read.is_reverse:
                yield read
    # prepare the output array and load the experiment
    bam = pysam.Samfile(path_to_bam, "rb") # load the bam file
    density_array = np.zeros((2,bam.lengths[refseq_index]), dtype)
    # load and filter the reads and add them to the density array
    filtered_reads = filterReads(bam.fetch(bam.references[refseq_index]))
    for read in filtered_reads:
        try:
            if read.is_read2: # maps to the minus strand
                density_array[1, read.pos] += 1
            else: # maps to the plus strand
                density_array[0, read.pos+read.template_length-1] += 1
        except: # some will map to a point beyond the extent of the genome - this is of course problematic!
            pass
    return density_array

In [7]:
# map all 5'-ends
##############
path_to_bam_folder = '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/'
output_folder_name = 'five_prime_numpy_all'
mapping_function = map5primeends_ezpool
kwargs = {'min_mapq':0, 'refseq_index':0}
##############

# get list of files in folder
stdout, _ = quickshell('ls %s*.bam'%path_to_bam_folder, print_output=False, return_output=True)
# regex out all bam files
bam_files = stdout.split('\n')[:-1]
# make the folder if it doesn't exist
quickshell('mkdir %s%s'%(path_to_bam_folder,output_folder_name), print_output=False, return_output=False)
# step through bam files
for bam_path in bam_files:
    numpy_file = mapping_function(bam_path, **kwargs)
    np.save('%s/%s/%s'%(path_to_bam_folder,output_folder_name,bam_path.split('/')[-1].replace('.bam','')), numpy_file)

In [8]:
# map full densities
##############
path_to_bam_folder = '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/'
output_folder_name = 'density_numpy_all'
mapping_function = mapfragdensity_ezpool
kwargs = {'min_mapq':0, 'refseq_index':0}
##############

# get list of files in folder
stdout, _ = quickshell('ls %s*.bam'%path_to_bam_folder, print_output=False, return_output=True)
# regex out all bam files
bam_files = stdout.split('\n')[:-1]
# make the folder if it doesn't exist
quickshell('mkdir %s%s'%(path_to_bam_folder,output_folder_name), print_output=False, return_output=False)
# step through bam files
for bam_path in bam_files:
    numpy_file = mapping_function(bam_path, **kwargs)
    np.save('%s/%s/%s'%(path_to_bam_folder,output_folder_name,bam_path.split('/')[-1].replace('.bam','')), numpy_file)

In [9]:
# map all 3'-ends
##############
path_to_bam_folder = '/home/pculviner/notebook/storage/rnaseq/PCe22_novaseq/ezpool_pipeline/'
output_folder_name = 'three_prime_numpy_all'
mapping_function = map3primeends_ezpool
kwargs = {'min_mapq':0, 'refseq_index':0}
##############

# get list of files in folder
stdout, _ = quickshell('ls %s*.bam'%path_to_bam_folder, print_output=False, return_output=True)
# regex out all bam files
bam_files = stdout.split('\n')[:-1]
# make the folder if it doesn't exist
quickshell('mkdir %s%s'%(path_to_bam_folder,output_folder_name), print_output=False, return_output=False)
# step through bam files
for bam_path in bam_files:
    numpy_file = mapping_function(bam_path, **kwargs)
    np.save('%s/%s/%s'%(path_to_bam_folder,output_folder_name,bam_path.split('/')[-1].replace('.bam','')), numpy_file)